In [1]:
import os
import copy
import json
import logging
import re

import torch
from torch.utils.data import TensorDataset, RandomSampler

from utils import get_labels
from KoCharELECTRA_master.tokenization_kocharelectra import KoCharElectraTokenizer
tokenizer = KoCharElectraTokenizer.from_pretrained('monologg/kocharelectra-base-discriminator')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ElectraTokenizer'. 
The class this function is called from is 'KoCharElectraTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ElectraTokenizer'. 
The class this function is called from is 'KoCharElectraTokenizer'.


### 데이터 text, label 분류

In [3]:
def df(path,name):
    texts = []
    labels=[]
    with open(os.path.join(path, name), 'r', encoding='utf-8') as f:
        for line in f:
            text, label = line.split('\t')
            label=label.split()
            texts.append(text)
            labels.append(label)
    return texts , labels
print('texts:',df('./data','train.tsv')[0][:10])
print('labels:',df('./data','train.tsv')[1][:10])

texts: ['오늘 삼성전자 주가는 얼마야', '두산에너빌리티 현황을 알려줘', '오늘 급등한 종목은 뭐야', '지금 LG전자 주가는 얼마야', '현재 삼성물산 주가는 얼마야', '오늘 한국카본 주가는 얼마야', '어제 LG화학 종가는 얼마니', '오늘 옵티시스 주가는 얼마야', '어제 대동기어 주가는 얼마야', '한화솔루션 가격을 알려줘']
labels: [['DAT-B', 'DAT-I', 'O', 'ST-B', 'ST-I', 'ST-I', 'ST-I', 'O', 'PR-B', 'PR-I', 'O', 'O', 'O', 'O', 'O'], ['ST-B', 'ST-I', 'ST-I', 'ST-I', 'ST-I', 'ST-I', 'ST-I', 'O', 'PR-B', 'PR-I', 'O', 'O', 'O', 'O', 'O'], ['DAT-B', 'DAT-I', 'O', 'O', 'O', 'O', 'O', 'ST-B', 'ST-I', 'O', 'O', 'O', 'O'], ['DAT-B', 'DAT-I', 'O', 'ST-B', 'ST-I', 'ST-I', 'ST-I', 'O', 'PR-B', 'PR-I', 'O', 'O', 'O', 'O', 'O'], ['DAT-B', 'DAT-I', 'O', 'ST-B', 'ST-I', 'ST-I', 'ST-I', 'O', 'PR-B', 'PR-I', 'O', 'O', 'O', 'O', 'O'], ['DAT-B', 'DAT-I', 'O', 'ST-B', 'ST-I', 'ST-I', 'ST-I', 'O', 'PR-B', 'PR-I', 'O', 'O', 'O', 'O', 'O'], ['DAT-B', 'DAT-I', 'O', 'ST-B', 'ST-I', 'ST-I', 'ST-I', 'O', 'PR-B', 'PR-I', 'O', 'O', 'O', 'O', 'O'], ['DAT-B', 'DAT-I', 'O', 'ST-B', 'ST-I', 'ST-I', 'ST-I', 'O', 'PR-B', 'PR-I', 'O', 'O', 'O', 'O', 

### 음절별로 id, label 붙히기

In [4]:
df_train=df('./data','train.tsv')[0]
df_t=[]
for i in df_train:
    df_t += i
df_label=df('./data','train.tsv')[1]
df_l=[]
for i in df_label:
    df_l += i
import pandas as pd
id=[]
for i in range(len(df_l)):
    id.append(i)
df_sum=pd.DataFrame({'id':id,'text':df_t,'label':df_l})
df_sum


,id,text,label
0,0,오,DAT-B
1,1,늘,DAT-I
2,2,,O
3,3,삼,ST-B
4,4,성,ST-I
...,...,...,...
3385,3385,,O
3386,3386,시,O
3387,3387,작,O
3388,3388,인,O


In [5]:
df_train=df('./data','train.tsv')
df_valid=df('./data','valid.tsv')
texts_train=df_train[0]
text_tokenized = tokenizer(texts_train, padding='max_length', max_length=128, truncation=True, return_tensors="pt")
labels_train=df_train[1]

import numpy as np

def text_to_label(text,label):
    ttls=[]
    for t,l in zip(text,label):
        ttl=[t,l]
        ttls.append(ttl)
    
    return ttls
     
a=text_to_label(texts_train[0], labels_train[0])
for i in a:
    print(i)


['오', 'DAT-B']
['늘', 'DAT-I']
[' ', 'O']
['삼', 'ST-B']
['성', 'ST-I']
['전', 'ST-I']
['자', 'ST-I']
[' ', 'O']
['주', 'PR-B']
['가', 'PR-I']
['는', 'O']
[' ', 'O']
['얼', 'O']
['마', 'O']
['야', 'O']


### 라벨 딕셔너리

In [6]:
# 개체라벨 딕셔너리
labels=[label.strip() for label in open(os.path.join('./data', 'label.txt'), 'r', encoding='utf-8')]
labels_to_ids = {word:i for i, word in enumerate(labels)}
ids_to_labels= {i:word for i, word in enumerate(labels)}
labels_to_ids

{'UNK': 0,
 'O': 1,
 'DAT-B': 2,
 'DAT-I': 3,
 'ST-B': 4,
 'ST-I': 5,
 'PR-B': 6,
 'PR-I': 7}

In [7]:
import torch
label_all_tokens=False
def align_label(text, label):
    tokenized_input= tokenizer(text, padding='max_length', max_length=32, truncation=True,return_tensors="pt")
    label_ids = []
    previous_word_idx=None
    label_all_tokens=True
    j=0
    for i in tokenized_input['input_ids'][0]:

            
        if i== 0:
            label_ids.append(-100)
        elif i== 2:
            label_ids.append(-100)
        elif i==3:
            label_ids.append(-100)
        else:
            label_ids.append(labels_to_ids[label[j]])
            j +=1
      

    return label_ids

class DataSequence(torch.utils.data.Dataset):

    def __init__(self,df):


        self.texts = [tokenizer(i,
                               padding='max_length', max_length = 32, truncation=True, return_tensors="pt") for i in df[0]]
                               
        self.labels = [align_label(i,j) for i,j in zip(df[0], df[1])]

    def __len__(self):

        return len(self.labels)

    def get_batch_data(self, idx):

        return self.texts[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels

In [8]:
df_train=df('./data','train.tsv')
DataSequence(df_train)[0]
len([align_label(i,j) for i,j in zip(df_train[0], df_train[1])])

311

In [9]:
from torch.utils.data import DataLoader
train_dataset = DataSequence(df_train)
val_dataset = DataSequence(df_valid)

train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=1, shuffle=True)
val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=4)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


In [10]:
from transformers import (
    BertConfig,
    DistilBertConfig,
    ElectraConfig,
    ElectraTokenizer,
    BertTokenizer,
    BertForTokenClassification,
    DistilBertForTokenClassification,
    AutoModelForTokenClassification,AutoTokenizer, ElectraForSequenceClassification, AdamW,ElectraModel,
    )

class Model(torch.nn.Module):

    def __init__(self):

        super(Model, self).__init__()

        self.bert = AutoModelForTokenClassification.from_pretrained("monologg/kocharelectra-base-discriminator", num_labels=len(labels))

    def forward(self, input_id, token_type_id,mask, label):

        output = self.bert(input_ids=input_id, token_type_ids=token_type_id,attention_mask=mask, labels=label, return_dict=False)

        return output

### 학습(파인튜닝)

In [11]:
from torch_optimizer import SGDP
from tqdm import tqdm, trange
from torch.utils.data import DataLoader

def train_loop(model, df_train, df_valid):

    train_dataset = DataSequence(df_train)
    val_dataset = DataSequence(df_valid)

    train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=16, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda")

    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()
    best_acc = 0
    best_loss = 0

    for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for train_data, train_label in tqdm(train_dataloader):

            train_label = train_label[0].to(device)
            token_type_id= train_data['token_type_ids'][0].to(device)
            mask = train_data['attention_mask'][0].to(device)
            input_id = train_data['input_ids'][0].to(device)

            optimizer.zero_grad()
            loss = model(input_id,token_type_id, mask, train_label)[0]
            logits= model(input_id, token_type_id,mask, train_label)[1]

            logits_clean = logits[0][train_label != -100]
            label_clean = train_label[train_label != -100]

            predictions = logits_clean.argmax(dim=1)

            acc = (predictions == label_clean).float().mean()
            total_acc_train += acc
            total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label[0].to(device)
            token_type_id= train_data['token_type_ids'][0].to(device)
            mask = val_data['attention_mask'][0].to(device)
            input_id = val_data['input_ids'][0].to(device)

            loss= model(input_id,token_type_id, mask, val_label)[0]
            logits=model(input_id,token_type_id, mask, val_label)[1]

            logits_clean = logits[0][val_label != -100]
            label_clean = val_label[val_label != -100]

            predictions = logits_clean.argmax(dim=1)          

            acc = (predictions == label_clean).float().mean()
            total_acc_val += acc
            total_loss_val += loss.item()

        val_accuracy = total_acc_val / len(val_dataloader)
        val_loss = total_loss_val / len(val_dataloader)

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(train_dataloader): .3f} | Accuracy: {total_acc_train / len(train_dataloader): .3f} | Val_Loss: {total_loss_val / len(val_dataloader): .3f} | Accuracy: {total_acc_val / len(val_dataloader): .3f}')

LEARNING_RATE = 1e-5
EPOCHS = 15
model=Model()
train_loop(model, df_train, df_valid)

Some weights of the model checkpoint at monologg/kocharelectra-base-discriminator were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/kocharelectra-base-discriminator and are newly initialized: ['classifi

Epochs: 1 | Loss:  1.850 | Accuracy:  0.429 | Val_Loss:  1.763 | Accuracy:  0.619


100%|██████████| 20/20 [00:01<00:00, 14.94it/s]


Epochs: 2 | Loss:  1.484 | Accuracy:  0.698 | Val_Loss:  1.607 | Accuracy:  0.409


100%|██████████| 20/20 [00:01<00:00, 15.05it/s]


...

Epochs: 13 | Loss:  0.465 | Accuracy:  0.914 | Val_Loss:  0.394 | Accuracy:  0.996


100%|██████████| 20/20 [00:01<00:00, 13.18it/s]


Epochs: 14 | Loss:  0.501 | Accuracy:  0.942 | Val_Loss:  0.327 | Accuracy:  0.993


100%|██████████| 20/20 [00:01<00:00, 15.02it/s]


Epochs: 15 | Loss:  0.369 | Accuracy:  0.948 | Val_Loss:  0.286 | Accuracy:  0.996


### 테스트 및 평가

In [12]:
df_test=df('./data','test.tsv')
test_dataset = DataSequence(df_test)

test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1)

for train_data, train_label in test_dataloader:
    train_label = train_label[0].to(device)
    token_type_id= train_data['token_type_ids'][0].to(device)
    mask = train_data['attention_mask'][0].to(device)
    input_id = train_data['input_ids'][0].to(device)
    print(model(input_id,token_type_id, mask, train_label)[0])
    logits_clean=model(input_id,token_type_id, mask, train_label)[1][0][train_label != -100]
   
df_test=df('./data','test.tsv')
def evaluate(model, df_test):

    test_dataset = DataSequence(df_test)

    test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0.0
    total_cor=0.0
    total_n=0.0

    for test_data, test_label in test_dataloader:

        test_label = test_label[0].to(device)
        token_type_id= train_data['token_type_ids'][0].to(device)
        mask = test_data['attention_mask'][0].to(device)
        input_id = test_data['input_ids'][0].to(device)
          
        loss= model(input_id, token_type_id,mask, test_label.long())[0]
        logits= model(input_id, token_type_id,mask, test_label.long())[1]

        logits_clean = logits[0][test_label != -100]
        label_clean = test_label[test_label != -100]

        predictions = logits_clean.argmax(dim=1)
        acc= (predictions == label_clean).float().mean()    
        print("label:", label_clean)
        print("pred:",predictions)
        total_acc_test += acc
    print(f'Test Accuracy: {total_acc_test / len(test_dataloader): .3f}')

evaluate(model, df_test)
   

tensor(0.2379, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2564, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3746, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2010, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0934, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3717, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3488, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1981, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7370, device='cuda:0', grad_fn=<NllLossBackward0>)
label: tensor([4, 5, 5, 5, 1, 6, 7, 1, 1, 1, 1, 1], device='cuda:0')
pred: tensor([4, 5, 5, 5, 1, 6, 7, 1, 1, 1, 1, 1], device='cuda:0')
label: tensor([4, 5, 5, 5, 5, 5, 5, 1, 6, 7, 1, 1, 1, 1, 1, 1], device='cuda:0')
pred: tensor([4, 5, 5, 5, 5, 5, 5, 1, 6, 7, 1, 1, 1, 1, 1, 1], device='cuda:0')
label: tensor([1, 1, 1, 6, 7, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
pred: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
label: tensor([4, 5

### 출력

In [13]:
def align_word_ids(text):
    tokenized_input= tokenizer(text, padding='max_length', max_length=32, truncation=True,return_tensors="pt")
    label_ids = []
    previous_word_idx=None
    label_all_tokens=True
    j=0
    for i in tokenized_input['input_ids'][0]:

            
        if i== 0:
            label_ids.append(-100)
        elif i== 2:
            label_ids.append(-100)
        elif i==3:
            label_ids.append(-100)
        else:
            label_ids.append(1)
            j +=1
      

    return label_ids

def evaluate_one_text(model, sentence):


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    text = tokenizer(sentence, padding='max_length', max_length = 32, truncation=True, return_tensors="pt")

    mask = text['attention_mask'][0].unsqueeze(0).to(device)
    token_type_id= train_data['token_type_ids'][0].to(device)
    input_id = text['input_ids'][0].unsqueeze(0).to(device)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)

    logits = model(input_id,token_type_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    print(sentence)
    print(prediction_label)
            
evaluate_one_text(model, '오늘 삼성죤자 주가 좀 알려줘')

오늘 삼성죤자 주가 좀 알려줘
['DAT-B', 'DAT-I', 'O', 'ST-B', 'ST-I', 'ST-I', 'ST-I', 'O', 'PR-B', 'PR-I', 'O', 'O', 'O', 'O', 'O', 'O']
